## PMML模型发布DEMO

以XGBoost为例，使用PMML文件进行模型发布。
首先，我们需要有一个XGBoost模型，为此，可以以Iris数据集训练一个简单的二分类模型（只用其中的两类）。
然后利用 XGBoost 训练得到模型文件。

利用XGBoost模型转PMML的工具：https://github.com/jpmml/jpmml-xgboost
可以非常容易进行转换。

In [1]:
from sklearn.datasets import load_iris

iris = load_iris()

In [2]:
X = iris.data
y = (iris.target == 1).astype(int)

In [3]:
import xgboost as xgb

params = {
    'objective' : 'reg:logistic',
    'num_round' : 10,
    'max_depth' : 3
}

dtrain = xgb.DMatrix(X, label=y)
evallist = [(dtrain, 'train')]
bst = xgb.train(params, dtrain, evals=evallist)
bst.save_model('xgb.bin')

[0]	train-rmse:0.378794
[1]	train-rmse:0.29549
[2]	train-rmse:0.241123
[3]	train-rmse:0.20289
[4]	train-rmse:0.177793
[5]	train-rmse:0.161685
[6]	train-rmse:0.150411
[7]	train-rmse:0.141292
[8]	train-rmse:0.136706
[9]	train-rmse:0.130609


/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### 生成特征映射文件
特征映射文件包括三列，第一列是id，第二列是特征名，第三列是特征数据类型.
参考 <https://github.com/jpmml/jpmml-xgboost/tree/master/src/test/resources/csv>。

数据类型参考：<https://github.com/jpmml/jpmml-xgboost/blob/4cc1d8e191b58dfde1c015cf3bc0e7ff3c80a2af/src/main/java/org/jpmml/xgboost/FeatureMap.java#L66>

In [4]:
f = open('fmap.txt', 'w')
for i, fn in enumerate(iris.feature_names):
    f.write('%d\t%s\t%s\n' % (i, fn, 'q')) 
f.close()

!cat fmap.txt

0	sepal length (cm)	q
1	sepal width (cm)	q
2	petal length (cm)	q
3	petal width (cm)	q


#### 转换模型
利用 jpmml-xgboost 项目<https://github.com/jpmml/jpmml-xgboost>提供的工具，进行转换。

你也可以直接下载我已经编译好的jar包 <https://github.com/tracholar/wiki/raw/master/src/machine-learning/converter-executable-1.2-SNAPSHOT.jar>。

然后执行下述命令，即可得到转换后的PMML文件 xgb.pmml.xml。

In [5]:
!java -jar converter-executable-1.2-SNAPSHOT.jar --model-input xgb.bin  --fmap-input fmap.txt  --pmml-output xgb.pmml.xml

In [6]:
!head -12 xgb.pmml.xml

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML xmlns="http://www.dmg.org/PMML-4_3" version="4.3">
	<Header>
		<Application name="JPMML-XGBoost" version="1.2-SNAPSHOT"/>
		<Timestamp>2017-10-18T03:53:12Z</Timestamp>
	</Header>
	<DataDictionary>
		<DataField name="_target" optype="continuous" dataType="float"/>
		<DataField name="sepal width (cm)" optype="continuous" dataType="float"/>
		<DataField name="petal length (cm)" optype="continuous" dataType="float"/>
		<DataField name="petal width (cm)" optype="continuous" dataType="float"/>
	</DataDictionary>
